# * ETL-Actual MKS : Broadband
    (P, G, H, HH, Z) level

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import oracledb
import re
import FN_Actual_MKS_Broadband as fn

config = configparser.ConfigParser()
config.read('../../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
next_week_dt = curr_dt + dt.timedelta(days=7)
str_curr_dt = curr_dt.strftime('%Y%m%d')
str_next_week_dt = next_week_dt.strftime('%Y%m%d')
yyyymm_curr = curr_dt.strftime('%Y%m')
yyyymm_next_week = next_week_dt.strftime('%Y%m')

### Lasted Source Summary

In [2]:
# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
# print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()


try:
    src_cur.execute("""
        SELECT TM_KEY_MTH
            , SUM(SUBS) TOTAL
            , SUM(CASE WHEN ISP = 'TOL' THEN SUBS END) TOL
            , SUM(CASE WHEN ISP = '3BB' THEN SUBS END) "3BB"
            , SUM(CASE WHEN ISP = 'AIS' THEN SUBS END) AIS
            , SUM(CASE WHEN ISP IN ('CAT', 'TOT') THEN SUBS END) NT
        FROM CORPNSBOX.FCT_BB_SHARE_SUBS_CCAATT A
        WHERE TM_KEY_MTH >= 202401
        --WHERE TM_KEY_MTH = (SELECT MAX(TM_KEY_MTH) FROM CORPNSBOX.FCT_BB_SHARE_SUBS_CCAATT NOLOCK)
        AND EXISTS (SELECT 1 FROM CDSAPPO.DIM_MOOC_AREA O
			        WHERE O.REMARK <> 'Dummy'
			        AND O.CCAATT = A.CCAATT)
        GROUP BY TM_KEY_MTH
        ORDER BY 1
    """)
    rows = src_cur.fetchall()
    # print(f'\nCurrent Source Summary...')
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    # chk_src_df['DATA'] = 'Source'
    mod_col_list = chk_src_df.iloc[:, 1:6].columns.tolist()
    # for col in mod_col_list:
    #     chk_src_df[col] = chk_src_df[col].apply(lambda x: format(x, ',.0f'))
    print(f'\n{chk_src_df}')


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    # print(f'\n{TDMDBPR_db} : Disconnected')


   TM_KEY_MTH         TOTAL      TOL           3BB           AIS            NT
0      202401  8.392163e+06  3062361  2.206741e+06  1.703305e+06  1.419756e+06
1      202402  8.399105e+06  3065282  2.197017e+06  1.716028e+06  1.420779e+06
2      202403  8.414203e+06  3072991  2.187215e+06  1.732168e+06  1.421829e+06
3      202404  8.422602e+06  3073168  2.177393e+06  1.749183e+06  1.422858e+06
4      202405  8.439795e+06  3082458  2.167565e+06  1.765884e+06  1.423888e+06


### Lasted Fact Summary

In [3]:
# Connect : AKPIPRD
tgt_dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
tgt_conn = oracledb.connect(tgt_dsn)
# print(f'\n{AKPIPRD_db} : Connected')
tgt_cur = tgt_conn.cursor()


try:
    tgt_cur.execute("""
        SELECT TM_KEY_MTH
            , SUM(CASE WHEN METRIC_CD IN ('VIN00025', 'VIN00026', 'VIN00027', 'VIN00028') THEN METRIC_VALUE END) TOTAL
            , SUM(CASE WHEN METRIC_CD = 'VIN00025' THEN METRIC_VALUE END) TOL
            , SUM(CASE WHEN METRIC_CD = 'VIN00026' THEN METRIC_VALUE END) "3BB"
            , SUM(CASE WHEN METRIC_CD = 'VIN00027' THEN METRIC_VALUE END) AIS
            , SUM(CASE WHEN METRIC_CD = 'VIN00028' THEN METRIC_VALUE END) NT
            --, MAX(LOAD_DATE) LOAD_DATE
        FROM AUTOKPI.FCT_BROADBAND_MKS NOLOCK
        WHERE TM_KEY_MTH >= 202401 --AND TM_KEY_MTH <= 202404
        --WHERE TM_KEY_MTH = 202403
        --WHERE TM_KEY_MTH = (SELECT MAX(TM_KEY_MTH) FROM AUTOKPI.FCT_BROADBAND_MKS NOLOCK WHERE REMARK IS NULL)
        AND TM_KEY_DAY LIKE '%01'
        AND AREA_TYPE = 'P'
        AND REMARK IS NULL
        GROUP BY TM_KEY_MTH
        ORDER BY 1
    """)
    rows = tgt_cur.fetchall()
    # print(f'\nCurrent Fact Summary...')
    chk_tgt_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in tgt_cur.description])
    # chk_tgt_df['DATA'] = 'Fact'
    mod_col_list = chk_tgt_df.iloc[:, 1:6].columns.tolist()
    # for col in mod_col_list:
    #     chk_tgt_df[col] = chk_tgt_df[col].apply(lambda x: format(x, ',.0f'))
    print(f'\n{chk_tgt_df}')


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    tgt_conn.close()
    # print(f'\n{AKPIPRD_db} : Disconnected')


   TM_KEY_MTH         TOTAL      TOL           3BB           AIS            NT
0      202401  8.392163e+06  3062361  2.206741e+06  1.703305e+06  1.419756e+06
1      202402  8.399105e+06  3065282  2.197017e+06  1.716028e+06  1.420779e+06
2      202403  8.414203e+06  3072991  2.187215e+06  1.732168e+06  1.421829e+06
3      202404  8.422602e+06  3073168  2.177393e+06  1.749183e+06  1.422858e+06
4      202405  8.439795e+06  3082458  2.167565e+06  1.765884e+06  1.423888e+06


### Check Diff Summary

In [4]:
chk_diff_df = chk_src_df.iloc[-1].compare(chk_tgt_df.iloc[-1])
chk_diff_df['diff'] = chk_diff_df['self'] - chk_diff_df['other']
mod_col_list = chk_diff_df.columns.tolist()
for col in mod_col_list:
    chk_diff_df[col] = chk_diff_df[col].apply(lambda x: format(x, ',.0f'))

chk_diff_df

,self,other,diff


### Input Parameter

In [5]:
# v_update_flag = 'Y' # Test

v_update_flag = 'Y' if chk_diff_df.size > 0 else 'N'
v_mth_end_src = chk_src_df['TM_KEY_MTH'].max().astype(float)
v_mth_end_fct = chk_tgt_df['TM_KEY_MTH'].max().astype(float)
v_mth_end_fct_year = pd.to_datetime(v_mth_end_fct, format='%Y%m').year
v_mth_end_fct_month = pd.to_datetime(v_mth_end_fct, format='%Y%m').month
v_prev_mth_fct = v_mth_end_fct-1 if v_mth_end_fct_month != 1 else float(str(v_mth_end_fct_year-1)+str(12))

print(f'v_update_flag: {v_update_flag}')
print(f'yyyymm_next_week: {yyyymm_next_week}')
print(f'v_mth_end_src: {v_mth_end_src}')
print(f'v_mth_end_fct: {v_mth_end_fct}')
print(f'v_prev_mth_fct: {v_prev_mth_fct}')

v_update_flag: N
yyyymm_next_week: 202407
v_mth_end_src: 202405.0
v_mth_end_fct: 202405.0
v_prev_mth_fct: 202404.0


## ETL Process...

### DB source to DB Target
    Delete -> Insert

    Source : CORPNSBOX.FCT_BB_SHARE_SUBS_CCAATT
             CDSAPPO.DIM_MOOC_AREA
             CDSAPPO.DIM_TIME
    
    Target : AUTOKPI.FCT_BROADBAND_MKS

In [6]:
# Create Param
v_target_schema = 'AUTOKPI'
v_target_table = 'FCT_BROADBAND_MKS'
v_sql_rawdata_fact = 'Raw-FCT_BROADBAND_MKS.sql'
v_sql_mockup_fact = 'Mock-FCT_BROADBAND_MKS.sql'

job_start_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nJob Start... {job_start_datetime}')


# Process flow
if v_update_flag == 'Y':
    if yyyymm_next_week > str(v_mth_end_fct):
        print(f'\n*** Source update & mockup to next month Fact ***')
        fn.src_update_to_fact(v_mth_end_fct, v_target_schema, v_target_table, v_sql_rawdata_fact)
    else:
        print(f'\n*** Source update current month Fact ***')
        fn.src_update_to_fact(v_prev_mth_fct, v_target_schema, v_target_table, v_sql_rawdata_fact)

elif v_update_flag == 'N':
    if yyyymm_next_week > str(v_mth_end_fct):
        print(f'\n*** Last Fact mockup to next month ***')
        fn.mockup_to_fact(v_mth_end_fct, v_target_schema, v_target_table, v_sql_mockup_fact)
    else:
        print(f'\n*** Not update ***')


Job Start... 2024-06-24, 12:25:00

*** Last Fact mockup to next month ***
mth_end_fct: 202405.0
target_schema: AUTOKPI
target_table: FCT_BROADBAND_MKS
sql_mockup_fact: Mock-FCT_BROADBAND_MKS.sql
v_param: {'mth_end_fct': 202405.0}

AKPIPRD : Connected

Processing...

   -> DELETE : "FCT_BROADBAND_MKS" : Done !

   -> INSERT : "FCT_BROADBAND_MKS" : Done !

AKPIPRD : Disconnected

Job Done !!!


In [7]:
''' Example DataFrame '''

# tmp_df = src_df.groupby(['VERSION', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME']).agg({'METRIC_VALUE': 'mean', 'TM_KEY_MTH': 'nunique', 'AREA_TYPE': 'nunique', 'AREA_CD': 'nunique'}).reset_index()
# tmp_df = src_df.groupby(['VERSION', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME']).agg({'TM_KEY_MTH': ['min','max'], 'METRIC_VALUE': 'mean', 'AREA_TYPE': 'nunique', 'AREA_CD': 'nunique'}).reset_index()
tmp_df = src_df.groupby(['VERSION', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME']).agg({'TM_KEY_DAY': ['min','max'], 'METRIC_VALUE': 'mean', 'AREA_TYPE': 'nunique', 'AREA_CD': 'nunique'}).reset_index()
tmp_df

NameError: name 'src_df' is not defined